# Video Capturing

In [1]:
import numpy as np
import cv2 as cv
import os
import PIL as ImageGrab
import pyautogui as py


# Image Processing

In [4]:
def takeSS():
    im = py.screenshot('image.png',region=(5,30, 847, 542))
    # Change the format that OpenCV can understand
    img = np.array(im)
    img_cvt = cv.cvtColor(img, cv.COLOR_RGB2BGR)

    # cv.imshow("das", img_cvt)
    # cv.waitKey(0)
    # cv.destroyAllWindows()
    return img_cvt

takeSS()

array([[[243, 243, 243],
        [243, 243, 243],
        [243, 243, 243],
        ...,
        [ 24,  24,  24],
        [ 24,  24,  24],
        [ 24,  24,  24]],

       [[243, 243, 243],
        [243, 243, 243],
        [243, 243, 243],
        ...,
        [ 24,  24,  24],
        [ 24,  24,  24],
        [ 24,  24,  24]],

       [[243, 243, 243],
        [243, 243, 243],
        [243, 243, 243],
        ...,
        [ 24,  24,  24],
        [ 24,  24,  24],
        [ 24,  24,  24]],

       ...,

       [[ 80, 148,  48],
        [ 79, 144,  49],
        [ 79, 146,  50],
        ...,
        [ 24,  24,  24],
        [ 24,  24,  24],
        [ 24,  24,  24]],

       [[ 80, 140,  52],
        [ 82, 142,  53],
        [ 82, 144,  51],
        ...,
        [ 24,  24,  24],
        [ 24,  24,  24],
        [ 24,  24,  24]],

       [[ 83, 140,  54],
        [ 84, 140,  52],
        [ 82, 140,  50],
        ...,
        [ 24,  24,  24],
        [ 24,  24,  24],
        [ 24,  24,  24]]

Edge detection using Canny Edge Detection 

In [13]:
def canny_edge_detection(img):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY )
    kernel = 5
    blur = cv.GaussianBlur(gray, (kernel, kernel),0)
    canny = cv.Canny(blur, 50, 150)
    return canny


Specify the region of interest

In [ ]:
def roi(img):
    height = img.shape[0]
    width = img.shape[1]
    
    # Road mask
    mask = np.zeros_like(img)
    car_mask = np.zeros_like(img)
    
    # Define the coordinates for the trapezoid region
    # Adjust these points based on the shape of the lane and the car's position
    roi_points = np.array([
    [((250), (250)),    # Top-left corner
     ((600), (250)),    # Top-right corner     
     ((800), (300)),    # Bottom-right corner
     ((50),  (300))]    # Bottom-left corner    
    ], np.int32)

     # HARDCODE
    # triangle = np.array([
    # car_triangle = np.array([
    #     [(int(width * 0.1), height),  # Bottom-left point
    #      (int(width * 0.5), int(height * 0.5)),  # Top-middle point
    #      (int(width * 0.9), height)]  # Bottom-right point
    # ], np.int32)

    # Fill the triangular region
    cv.fillPoly(mask, roi_points, 255)
    #cv.fillPoly(mask, [car_triangle],0)
    
    # Use bitwise_and to apply the mask
    masked_image = cv.bitwise_and(img, mask)
    
    cv.imshow("ROI Applied", masked_image)
    cv.waitKey(0)
    cv.destroyAllWindows()
    
    return masked_image

# Example usage:
img_cvt = takeSS()  # Capture screenshot and convert
edges = canny_edge_detection(img_cvt)  # Perform Canny edge detection
masked_image = ros(edges)

print(img_cvt.shape)

cv.imshow("Masked Image", masked_image)
cv.waitKey(0)
cv.destroyAllWindows()


# For future adjustment, use trapezoid instead of triangle

In [276]:
def draw_road_roi(img):
    height, width = img.shape[0], img.shape[1]

    # Define a triangle that captures the road area in front of the car
    # triangle = np.array([
    #     [(int(width * 0.1), height),           # Bottom-left: 10% from the left of the frame
    #      (int(width * 0.5), int(height * 0.6)), # Top-middle: center of the frame, 60% up
    #      (int(width * 1.2), height)]           # Bottom-right: 90% from the left of the frame
    # ], np.int32)
    # HARDCODE
    roi_points = np.array([
    [((250), (250)),    # Top-left corner
     ((600), (250)),    # Top-right corner     
     ((800), (300)),    # Bottom-right corner
     ((50),  (300))]    # Bottom-left corner    
    ], np.int32)



    # Draw the triangle on the image for visualization
    cv.polylines(img, [roi_points], isClosed=True, color=(0, 255, 0), thickness=2)
    return img

# Example usage:
img_cvt = takeSS()  # Capture screenshot and convert
img_with_triangle = draw_road_triangle(img_cvt)

cv.imshow("Road Roi", img_with_triangle)
cv.waitKey(0)
cv.destroyAllWindows()

Coordinate Explanation:
* (int(width * 0.1), height) defines the bottom-left point of the triangle (10% of the frame width from the left, and at the bottom).
* (int(width * 0.5), int(height * 0.5)) is the top-middle point of the triangle (center of the width, and 50% down from the top).
* (int(width * 0.9), height) is the bottom-right point of the triangle (90% of the frame width from the left, and at the bottom).
How to Adjust the Triangle:
Visually Inspect the Lane:

Run the program and see where the triangle is placed. 
1. Adjust the width * 0.1 and width * 0.9 values to move the triangle left or right.
2. Adjust height * 0.5 to raise or lower the top point of the triangle.

Define the HLD (Hough Line Detection)

In [21]:
# Detect lines using Hough Transform
def houghlines(img):
    hough_lines = cv.HoughLinesP(img, 2, np.pi/180, 100, np.array([]), minLineLength = 40, maxLineGap =10)
    return hough_lines

In [35]:
# lines are made from points in openCV
def make_points(img, lineSI):
    slope, intercept = lineSI
    height = img.shape[0]
    y1 = int(height)
    y2 = int(y1*3.0/2)

    x1 = int((y1-intercept)/slope)
    x2 = int((y2-intercept)/slope)

    return [[x1, x2, y1, y2]]
# Before do this below code, you need to make sure that the line of the lane and the car
# is already correct. Specify the correct region of ineterest first
def average_slope_intercepts(img, lines):
    left_fit = []
    right_fit = []
    for line in lines:
        for x1, x2, y1, y2 in line:
            fit = np.polyfit((x1, x2), (y1, y2), 1)
            slope = fit[0]
            intercept = fit[1]

        if slope<0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))

    left_fit_average = np.average(left_fit, axis=0)
    right_fit_average = np.average(right_fit, axis=0)

    left_line = make_points(img, left_fit_average)
    right_line = make_points(img, right_fit_average)
    average_lines = [left_line, right_line]

    return average_lines


# Draw the lines on the image
def display_lines_average(img, lines):
    line_image = np.zeros_like(img)  # Create an empty image to draw lines on
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                # Draw the lines (red color, thickness = 10)
                cv.line(img, (x1, y1), (x2, y2), (0, 0, 255), 10)
    return img



# Run the detection
img_cvt = takeSS()  # Capture screenshot and convert
edges = canny_edge_detection(img_cvt)  # Perform Canny edge detection
h_lines = houghlines(edges)

# Call the line function
avg_lines = average_slope_intercepts(img_cvt, h_lines)
print(avg_lines)
print(h_lines)

lines_image = display_lines_average(img_cvt, avg_lines)
cv.imshow("Detected Lines", lines_image)
cv.waitKey(0)
cv.destroyAllWindows()

[[[179, 141, 542, 813]], [[410, 588, 542, 813]]]
[[[464 217 517 217]]

 [[299 196 355 196]]

 [[460 221 582 240]]

 [[745 277 799 287]]

 [[  1 271  58 263]]

 [[487 199 549 199]]

 [[552 215 608 215]]

 [[  3 267  71 258]]

 [[403 220 459 220]]

 [[265 208 311 208]]

 [[474 201 620 203]]

 [[246 211 313 211]]

 [[719 207 761 205]]

 [[660 301 846 365]]

 [[274 234 337 223]]

 [[715 215 769 216]]

 [[103 250 259 229]]

 [[325 195 373 195]]

 [[655 262 844 299]]

 [[ 63 266 161 250]]

 [[673 263 786 281]]

 [[529 229 737 259]]

 [[ 73 289 277 241]]

 [[704 269 841 291]]

 [[ 16 206  58 206]]

 [[134 185 193 193]]

 [[  0 326 317 235]]

 [[463 234 841 372]]

 [[196 203 252 202]]

 [[ 42 310 265 247]]

 [[240 203 313 200]]

 [[797 150 846 146]]

 [[609 195 753 151]]

 [[168 245 212 241]]

 [[706 260 786 272]]

 [[772 144 814 136]]

 [[461 197 507 196]]

 [[584 219 825 219]]

 [[134 211 177 210]]

 [[321 200 388 202]]

 [[399 168 442 185]]

 [[381 217 423 216]]

 [[560 236 614 256]]

 [[27

def average_slope_intercepts(img, lines):

sometimes(in most cases actually) it detects multiple lines.  to overcome this, we can use the average value within the lines as you’ve seen in this code.

bear in minds that, the lines (you can see at the ros function) is a lot of them. So you need to specify that only certain lines that is captured, hence it is called region of interest.

(542, 847, 3)


In [6]:
import time
import pyautogui as py
import pydirectinput

# Function to click at specified coordinates (using PyAutoGUI)
def click(x, y):
    py.click(x, y)  # Click at the specified coordinates
    time.sleep(0.5)  # Wait briefly to ensure the game is focused
    
def accelerate(hold_time):
    start = time.time()
    while time.time() - start < hold_time:
        pydirectinput.keyDown('up')
    pydirectinput.keyUp('up')

def turnLeft(hold_time):
    start = time.time()
    while time.time() - start < hold_time:
        pydirectinput.keyDown('a')
    pydirectinput.keyUp('a')

def turnRight(hold_time):
    start = time.time()
    while time.time() - start < hold_time:
        pydirectinput.keyDown('d')
    pydirectinput.keyUp('d')

# Accelerate for 10 seconds, then turn left and right
click(409, 275)
accelerate(10)
turnLeft(0.3)
turnRight(0.3)